## 1. Setup & Data Preparation

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd

In [2]:
data_dir = "UTKFace"
model_path = "models"

In [3]:
# Define the dataset class for age estimation
class UTKFaceDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        age = int(self.annotations.iloc[idx, 1])
        
        if self.transform:
            image = self.transform(image)

        return image, age

In [ ]:


# Define the transformation

train_transforms  = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the dataset
train_dataset = UTKFaceDataset(
    csv_file= "AgeDataset/train.csv", 
    root_dir=data_dir,
    transform=train_transforms
)

test_dataset = UTKFaceDataset(
    csv_file="AgeDataset/test.csv",
    root_dir=data_dir,
    transform=test_transforms
)


# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



## Training and evaluating

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from tqdm import tqdm

# Evaluate function, calculate the mean absolute error (MAE) for age estimation
def evaluate_model(model, test_loader):
    model.eval()
    total_mae = 0
    total_samples = 0
    with torch.no_grad():
        for images, ages in tqdm(test_loader, desc="Evaluating"):
            images = images.to(device)
            ages = ages.to(device)

            outputs = model(images)
            predicted_ages = outputs.squeeze().cpu().numpy()
            ages = ages.cpu().numpy()

            total_mae += np.sum(np.abs(predicted_ages - ages))
            total_samples += len(ages)
    mean_mae = total_mae / total_samples
    print(f"Mean Absolute Error (MAE): {mean_mae:.2f}")
    
    
    
# Train the model
def train_model(model, train_loader, optimizer, criterion, scheduler, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, ages in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images = images.to(device)
            ages = ages.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), ages.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        scheduler.step()
        evaluate_model(model, test_loader)
        
        
 


        

In [8]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride,
                                   padding=padding, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1)
      
    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

# Build Separable Convolutional Neural Network for gender classification
class GenderClassifierCnn(nn.Module):
    def __init__(self):
        super(GenderClassifierCnn, self).__init__()
        # Depthwise separable convolutional layers
        self.conv1 = DepthwiseSeparableConv(3, 32)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = DepthwiseSeparableConv(32, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = DepthwiseSeparableConv(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 2)
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)
        
        
    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        # Output layer
        x = self.fc2(x)
        return x
        



In [17]:
# CNN model for age estimation
class AgeEstimatorCnn(nn.Module):
    def __init__(self):
        super(AgeEstimatorCnn, self).__init__()
        # Depthwise separable convolutional layers
        self.conv1 = DepthwiseSeparableConv(3, 32)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = DepthwiseSeparableConv(32, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = DepthwiseSeparableConv(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = DepthwiseSeparableConv(128, 256)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = DepthwiseSeparableConv(256, 256)
        self.bn5 = nn.BatchNorm2d(256)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 4 * 4, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 512)
        self.bn_fc2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 1)
        
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn_fc2(self.fc2(x)))
        x = self.dropout(x)
        # Output layer
        x = self.fc3(x)
        
        return x
    
    

In [18]:
# Initialize the model, loss function, optimizer, and learning rate scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AgeEstimatorCnn().to(device)
criterion = nn.L1Loss()

optimizer = optim.Adam(model.parameters(), lr=0.0025, weight_decay=1e-4)  
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce learning rate every 5 epochs


In [19]:
# Train the model
train_model(model, train_loader, optimizer, criterion, scheduler, num_epochs=20)

Epoch 1/20: 100%|██████████| 652/652 [03:44<00:00,  2.90it/s]


Epoch [1/20], Loss: 15.0022


Evaluating: 100%|██████████| 131/131 [00:31<00:00,  4.22it/s]


Mean Absolute Error (MAE): 10.52


Epoch 2/20: 100%|██████████| 652/652 [03:28<00:00,  3.13it/s]


...

Mean Absolute Error (MAE): 5.30


Epoch 20/20: 100%|██████████| 652/652 [02:57<00:00,  3.67it/s]


Epoch [20/20], Loss: 8.4162


Evaluating: 100%|██████████| 131/131 [00:29<00:00,  4.37it/s]

Mean Absolute Error (MAE): 5.30


In [72]:
# Save the CNN model
torch.save(model.state_dict(), os.path.join(model_path, "age_estimator_cnn.pth"))

In [9]:
# CNN model with CBAM block for age estimation
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveAvgPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        
        return self.sigmoid(out) * x 
    
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out = torch.max(x, dim=1, keepdim=True).values
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x_cat)) * x
    
class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=8, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size=kernel_size)
    
    def forward(self, x):
        x = self.ca(x)
        x = self.sa(x)
        
        return x
    
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1) # Calculate the average weight of each channel
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        b, c, _, _ = x.size() # Batch size and number of channels
        y = self.global_avg_pool(x).view(b, c) # [B, C]
        y = self.fc(y).view(b, c, 1, 1) # [B, C, 1, 1]
        
        return x * y.expand_as(x)  # Scaled channel wise

In [10]:
# CNN model with CBAM block for age estimation
class CBAM_CNN(nn.Module):
    def __init__(self):
        super(CBAM_CNN, self).__init__()
        # Depthwise separable convolutional layers
        self.conv1 = DepthwiseSeparableConv(3, 32)
        self.bn1 = nn.BatchNorm2d(32)
        self.cbam1 = CBAM(32)
        
        self.conv2 = DepthwiseSeparableConv(32, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.cbam2 = CBAM(64)
        
        self.conv3 = DepthwiseSeparableConv(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        self.cbam3 = CBAM(128)
        
        self.conv4 = DepthwiseSeparableConv(128, 256)
        self.bn4 = nn.BatchNorm2d(256)
        self.cbam4 = CBAM(256)
        
        self.conv5 = DepthwiseSeparableConv(256, 256)
        self.bn5 = nn.BatchNorm2d(256)
        self.cbam5 = CBAM(256)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 4 * 4, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 512)
        self.bn_fc2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 1)
        
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(self.cbam1(F.relu(self.bn1(self.conv1(x)))))
        x = self.pool(self.cbam2(F.relu(self.bn2(self.conv2(x)))))
        x = self.pool(self.cbam3(F.relu(self.bn3(self.conv3(x)))))
        x = self.pool(self.cbam4(F.relu(self.bn4(self.conv4(x)))))
        x = self.pool(self.cbam5(F.relu(self.bn5(self.conv5(x)))))

        # Flatten the tensor
        x = x.view(x.size(0), -1)

        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)

        x = F.relu(self.bn_fc2(self.fc2(x)))
        x = self.dropout(x)

        # Output layer
        x = self.fc3(x)
        return x

In [87]:
# Initialize the CBAM model, loss function, optimizer, and learning rate scheduler
cbam_model = CBAM_CNN().to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(cbam_model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce learning rate every 5 epochs


In [88]:
# Train the CBAM model
train_model(cbam_model, train_loader, optimizer, criterion, scheduler, num_epochs=20)

Epoch 1/20: 100%|██████████| 652/652 [02:59<00:00,  3.63it/s]


Epoch [1/20], Loss: 25.5899


Evaluating: 100%|██████████| 131/131 [00:31<00:00,  4.22it/s]


Mean Absolute Error (MAE): 18.83


Epoch 2/20: 100%|██████████| 652/652 [02:59<00:00,  3.64it/s]


...

Mean Absolute Error (MAE): 7.49


Epoch 20/20: 100%|██████████| 652/652 [02:56<00:00,  3.69it/s]


Epoch [20/20], Loss: 10.4985


Evaluating: 100%|██████████| 131/131 [00:30<00:00,  4.31it/s]

Mean Absolute Error (MAE): 7.48


In [89]:
torch.save(cbam_model.state_dict(), os.path.join(model_path, "cbam_age_estimator.pth"))

In [11]:
# CNN model with Squeeze-and-Excitation (SE) and Spatial Attention for age estimation
class SE_SpatialAttention_CNN(nn.Module):
    def __init__(self):
        super(SE_SpatialAttention_CNN, self).__init__()
        # Depthwise separable convolutional layers
        self.conv1 = DepthwiseSeparableConv(3, 32)
        self.bn1 = nn.BatchNorm2d(32)
        self.se1 = SEBlock(32)
        self.spatial_att1 = SpatialAttention(kernel_size=7)
        
        self.conv2 = DepthwiseSeparableConv(32, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.se2 = SEBlock(64)
        self.spatial_att2 = SpatialAttention(kernel_size=7)
        
        self.conv3 = DepthwiseSeparableConv(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        self.se3 = SEBlock(128)
        self.spatial_att3 = SpatialAttention(kernel_size=7)
        
        self.conv4 = DepthwiseSeparableConv(128, 256)
        self.bn4 = nn.BatchNorm2d(256)
        self.se4 = SEBlock(256)
        self.spatial_att4 = SpatialAttention(kernel_size=7)
        
        self.conv5 = DepthwiseSeparableConv(256, 256)
        self.bn5 = nn.BatchNorm2d(256)
        self.se5 = SEBlock(256)
        self.spatial_att5 = SpatialAttention(kernel_size=7)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 4 * 4, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 512)
        self.bn_fc2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 1)
        
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(self.spatial_att1(self.se1(F.relu(self.bn1(self.conv1(x))))))
        x = self.pool(self.spatial_att2(self.se2(F.relu(self.bn2(self.conv2(x))))))
        x = self.pool(self.spatial_att3(self.se3(F.relu(self.bn3(self.conv3(x))))))
        x = self.pool(self.spatial_att4(self.se4(F.relu(self.bn4(self.conv4(x))))))
        x = self.pool(self.spatial_att5(self.se5(F.relu(self.bn5(self.conv5(x))))))

        # Flatten the tensor
        x = x.view(x.size(0), -1)

        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        
        x = F.relu(self.bn_fc2(self.fc2(x)))
        x = self.dropout(x)
        # Output layer
        x = self.fc3(x)
        return x

In [12]:
# Initialize the SE and Spatial Attention model, loss function, optimizer, and learning rate scheduler
se_spatial_model = SE_SpatialAttention_CNN().to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(se_spatial_model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce learning rate every 5 epochs


In [92]:
# Train the se_spatial_model
train_model(se_spatial_model, train_loader, optimizer, criterion, scheduler, num_epochs=20)

Epoch 1/20: 100%|██████████| 652/652 [02:55<00:00,  3.72it/s]


Epoch [1/20], Loss: 25.2194


Evaluating: 100%|██████████| 131/131 [00:29<00:00,  4.41it/s]


Mean Absolute Error (MAE): 18.79


Epoch 2/20: 100%|██████████| 652/652 [02:54<00:00,  3.73it/s]


...

Mean Absolute Error (MAE): 7.67


Epoch 20/20: 100%|██████████| 652/652 [02:54<00:00,  3.73it/s]


Epoch [20/20], Loss: 10.7563


Evaluating: 100%|██████████| 131/131 [00:29<00:00,  4.41it/s]

Mean Absolute Error (MAE): 7.67


In [93]:
# Save the se_spatial_model
torch.save(se_spatial_model.state_dict(), os.path.join(model_path, "se_spatial_model.pth"))

In [13]:
from torchvision.models import resnet50

# Transfer learning with pre-trained ResNet18 and CBAM Block for age estimation
class ResNetCBAM(nn.Module):
    def __init__(self, num_classes=1):
        super(ResNetCBAM, self).__init__()
        base_model = resnet50(pretrained=True)
        self.features = nn.Sequential(*list(base_model.children())[:-1])  # remove final fc
        self.cbam = CBAM(2048)
      
        self.classifier = nn.Sequential(
            nn.Linear(2048, 4096),
            nn.ReLU(),
            nn.BatchNorm1d(4096),
            nn.Dropout(0.5),

            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            
            nn.Linear(1024, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.cbam(x)
        
        x = x.view(x.size(0), -1)
        
        
        x = self.classifier(x)
        return x


In [14]:

# Initialize the ResNetCBAM model, loss function, optimizer, and learning rate scheduler
resnet_cbam_model = ResNetCBAM().to(device)


# Freezing the layers of ResNet18
for param in resnet_cbam_model.features.parameters():
    param.requires_grad = False
# Unfreeze the CBAM block and classifier layers
for param in resnet_cbam_model.cbam.parameters():
    param.requires_grad = True
for param in resnet_cbam_model.classifier.parameters():
    param.requires_grad = True
    
learnable_params = filter(lambda p: p.requires_grad, resnet_cbam_model.parameters())

criterion = nn.L1Loss()
optimizer = optim.Adam(learnable_params ,lr=0.0001, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce learning rate every 5 epochs


d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
# Train the ResNetCBAM model
train_model(resnet_cbam_model, train_loader, optimizer, criterion, scheduler, num_epochs=20)

Epoch 1/20: 100%|██████████| 652/652 [02:58<00:00,  3.66it/s]


Epoch [1/20], Loss: 31.0675


Evaluating: 100%|██████████| 131/131 [00:34<00:00,  3.84it/s]


Mean Absolute Error (MAE): 24.36


Epoch 2/20: 100%|██████████| 652/652 [02:57<00:00,  3.68it/s]


...

Mean Absolute Error (MAE): 3.16


Epoch 20/20: 100%|██████████| 652/652 [03:31<00:00,  3.08it/s]


Epoch [20/20], Loss: 8.1582


Evaluating: 100%|██████████| 131/131 [00:36<00:00,  3.59it/s]

Mean Absolute Error (MAE): 3.16


In [16]:
# SAve the ResNetCBAM model
torch.save(resnet_cbam_model.state_dict(), os.path.join(model_path, "resnet_cbam_age_estimator.pth"))

In [17]:
from torchvision.models import vgg16

# Transfer learning with pre-trained VGG16 and CBAM Block for age estimation
class VGG16CBAM(nn.Module):
    def __init__(self, num_classes=1):
        super(VGG16CBAM, self).__init__()
        base_model = vgg16(pretrained=True)
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        self.cbam = CBAM(512)
        
        self.classifier = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),

            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            
            nn.Linear(2048, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.cbam(x)
        x = F.adaptive_avg_pool2d(x, (1, 1)).view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [18]:
    
# Initialize the VGG16CBAM model, loss function, optimizer, and learning rate scheduler
vgg16_cbam_model = VGG16CBAM().to(device)
# Freezing the layers of VGG16
for param in vgg16_cbam_model.features.parameters():
    param.requires_grad = False
# Unfreeze the CBAM block and classifier layers
for param in vgg16_cbam_model.cbam.parameters():
    param.requires_grad = True
for param in vgg16_cbam_model.classifier.parameters():
    param.requires_grad = True
learnable_params = filter(lambda p: p.requires_grad, vgg16_cbam_model.parameters())
criterion = nn.L1Loss()
optimizer = optim.Adam(learnable_params, lr=0.0001, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce learning rate every 5 epochs

d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
# Train the VGG16CBAM model
train_model(vgg16_cbam_model, train_loader, optimizer, criterion, scheduler, num_epochs=20)

Epoch 1/20: 100%|██████████| 652/652 [03:00<00:00,  3.60it/s]


Epoch [1/20], Loss: 30.6390


Evaluating: 100%|██████████| 131/131 [00:33<00:00,  3.90it/s]


Mean Absolute Error (MAE): 22.35


Epoch 2/20: 100%|██████████| 652/652 [02:53<00:00,  3.75it/s]


...

Mean Absolute Error (MAE): 3.09


Epoch 20/20: 100%|██████████| 652/652 [03:23<00:00,  3.20it/s]


Epoch [20/20], Loss: 7.9713


Evaluating: 100%|██████████| 131/131 [00:36<00:00,  3.63it/s]

Mean Absolute Error (MAE): 3.09


In [20]:

torch.save(vgg16_cbam_model.state_dict(), os.path.join(model_path, "vgg16_cbam_age_estimator.pth"))